In [0]:
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys, random, os

drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/capstone_code")
sys.path.append("/content/drive/My Drive/capstone_code/p_file")

train = pd.read_csv("./training_set.csv").reset_index(drop=True)
test = pd.read_csv("./test_set.csv").reset_index(drop=True)

In [0]:
def load_data():
    train = pd.read_csv("./training_set.csv")
    test = pd.read_csv("./test_set.csv")
    from preprocessing.serialization import serialize_data_encoding
    from models.data_generator import data_generator_en

    train_input, train_label =  serialize_data_encoding(train, save=True, save_path="./")
    val_input, val_label =  serialize_data_encoding(test, save=True, save_path="./")

    train_generator = data_generator_en(train_input, train_label, batch_size=64)
    val_generator = data_generator_en(val_input, val_label, batch_size=64, augmented=False)  
    return train_generator, val_generator

def train_model_org(save_path, n_epoch=100, model_type = "contract", latent_dim=120, n_latent_layer=3, save_train_stats=True):
      from models.construction import Model_expand, Model_contract
      train_generator, val_generator = load_data()
      
      if model_type == "contract":
          print('Training model, latent dim = 20')
          model = Model_contract(save_dir=save_path)
          history = model.training_generator(train_generator, val_generator, 500, 200, n_epoch=n_epoch)
          print('Finish training.')
      if model_type == 'expand':
          print('Training model, latent dim = {}'.format(latent_dim))
          model = Model_expand(save_dir=save_path, latent_dim=latent_dim, n_latent_layer=n_latent_layer, early_stop=False)
          history = model.training_generator(train_generator, val_generator, 500, 200, n_epoch=n_epoch)
          print('Finish training.')
      if save_train_stats: 
          np.array(history.history['mean_absolute_error']).dump(save_path+"en_h1_tmae")
          np.array(history.history['val_mean_absolute_error']).dump(save_path+"en_h1_vmae")
          np.array(history.history['loss']).dump(save_path+"en_h1_tmse")
          np.array(history.history['val_loss']).dump(save_path+"en_h1_vmse")
      return model, history

In [0]:
sp = "./contract_20"

train_model_org(sp, model_type='contract')

In [0]:
sp = "./expand_80"

train_model_org(sp, model_type='expand', latent_dim=80, n_latent_layer=2)

In [0]:
sp = "./expand_120"

train_model_org(sp, model_type='expand', latent_dim=120, n_latent_layer=3)

In [0]:
sp = "./expand_160"

train_model_org(sp, model_type='expand', latent_dim=160, n_latent_layer=4)